In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 tensorflow-hub opencv-python matplotlib

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [ ]:
file_name = 'Seth_Day1_Converted.mp4'
window_name = "window"
interframe_wait_ms = 30

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [ ]:
Hip_Angles = []
Knee_Angles = []
Ankle_Angles = []

In [ ]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [ ]:
#[nose, left eye, right eye, left ear, right ear,
#left shoulder, right shoulder, left elbow, right elbow,
#left wrist, right wrist, left hip, right hip, left knee,
#right knee, left ankle, right ankle].
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [ ]:
cap = cv2.VideoCapture(file_name)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Reached end of video, exiting.")
        break
        
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
     # Extract landmarks
    try:
        
        # Get coordinates
        RShoulder = [keypoints_with_scores[0,6,0],keypoints_with_scores[0,6,1]]
        RHip = [keypoints_with_scores[0,12,0],keypoints_with_scores[0,12,1]]
        RKnee = [keypoints_with_scores[0,14,0],keypoints_with_scores[0,14,1]]
        RAnkle = [keypoints_with_scores[0,16,0],keypoints_with_scores[0,16,1]]
        
            
        # Calculate angle
        Hip_Angle = calculate_angle(RShoulder, RHip, RKnee)
        Knee_Angle = calculate_angle(RHip, RKnee, RAnkle)
        Hip_Angles.append(Hip_Angle)
        Knee_Angles.append(Knee_Angle)
                
        # Visualize angle
        cv2.putText(image, str(Hip_Angle), 
                        tuple(np.multiply(RHip, [1920, 1080]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (284, 384, 384), 2, cv2.LINE_AA
                            )
        # Visualize angle
        cv2.putText(image, str(Knee_Angle), 
                        tuple(np.multiply(RKnee, [1920, 1080]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                            )
                       
    except:
        pass
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)    
 
    
    cv2.imshow(window_name, frame)
    if ret == True:
        cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,
                cv2.WINDOW_FULLSCREEN)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
plt.plot(Hip_Angles)
plt.show()

In [ ]:
plt.plot(Knee_Angles)
plt.show()